In [1]:
import hashlib
import random
import time
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import re
import queue
import os 

# An adjusted version of BSE, so it's stochastic processes can be seeded and therefore be determinisitic. 
import BSE


class Block:
    def __init__(self, data=None, previous_hash=None, message=None, user_config=None, proof=None, seed=None):
        self.timestamp = time.time()
        self.data = data if data is not None else "Genesis Block"
        self.previous_hash = previous_hash if previous_hash is not None else "0"
        self.message = message if message is not None else "Genesis, message"
        self.nonce = None
        self.block_param = (str(self.timestamp) + str(self.data) + str(self.previous_hash) + str(self.message))
        self.hash = self.calculate_hash()
        self.market_time = None
        
        # Storing the user's configuration, proof, and seed for later validation purposes.
        # A miner must specify the parameters needed to validate their proof, when they are allowed to publish a block. 
        self.user_config = user_config
        self.proof = proof
        self.seed = seed
        


    def calculate_hash(self):
        # Include all parameters and the nonce in the hash calculation. 
        return hashlib.sha256((self.block_param + str(self.nonce)).encode('utf-8')).hexdigest()


    def calculate_seed(self):
    # Compute a new seed based on the hash and other parameters, which will be used in the pre-hashhing step. 
        return hashlib.sha256((self.block_param + str(self.nonce) + self.hash).encode('utf-8')).hexdigest()

    

    def pre_hash(self, target):
        # Updating timestamp and block_param for each iteration. 
        self.timestamp = time.time()
        self.block_param = (str(self.timestamp) + str(self.data) + str(self.previous_hash) + str(self.message))

        # Initialise nonce / random number taken from the below uniforkm distribution. 
        self.nonce = random.randint(0, 2**32)

        # Calculate initial hash. 
        self.hash = self.calculate_hash()
        
        # This is esentially a small PoW hash puzzle. 
        while int(self.hash[:target], 16) != 0:  # Check if the first 'target' characters are zeros.
            self.nonce = random.randint(0, 2**32)  # Sample a new nonce randomly from a uniform distribution.
            self.hash = self.calculate_hash()  # Recalculate hash with new nonce.

        seed = int(self.hash, 16)
        seed = seed % (2**32)

        return seed # This seed will be used to make market session outcomes determinisitc. 

    
    

    def post_hash(self, strat_file_path, seed, pouw_difficulty, verifying=False):
        
        # Skip the PPS threshold check for the validation case. 
        # When verifying, you don't need to check PPS if the proof itself is valid. 
        if not verifying:
            
            # Read the file into a df. 
            df = pd.read_csv(strat_file_path, sep="\t", header=None)

            # Splitt the single string in the cell into a list of values. 
            data_values = df.iloc[0, 0].split(', ')

            # Convert the list of values into df with separate cells. 
            df_parsed = pd.DataFrame([data_values]).transpose()

            # Identify rows with capital letters (trader types, nothign else is capital in the avg file). 
            capital_rows = [row[0] for row in df_parsed.itertuples(index=False) if isinstance(row[0], str) and row[0].isupper()]

            # Extract total profit values based on the trader types. 
            total_profit = sum([float(df_parsed.iloc[df_parsed.index[df_parsed[0] == trader_id].tolist()[0] + 3][0]) for trader_id in capital_rows])
            print('')
            print(f'The total profit per trader type in this market session is: {total_profit}')

            # Find average PPS, this is also referred to as aggregate PPS in the paper. 
            average_pps = total_profit / self.market_time
            print(f'The average pps across this market session across all traders is: {average_pps}')
            
            # For the time being, this is arbitrary because of lack of network functionaltiy. This is elaborated on in the paper. 
            threshold = pouw_difficulty
            if average_pps < threshold:
                print(f"Average PPS of: {average_pps:.2f} is below the threshold: {threshold:.2f}.")
            else:
                print(f"Average PPS of: {average_pps:.2f} is above the threshold: {threshold:.2f} for seed {seed}.")

            return average_pps > threshold
        else:
            return True
    
    


    def extract_proof(self, strat_file_path):
        
        # Check if the file exists and is not empty. 
        if not os.path.exists(strat_file_path) or os.path.getsize(strat_file_path) == 0:
            print(f"Error: File {strat_file_path} either doesn't exist or is empty.")
            return None
        
        # Read the csv with no header. 
        df = pd.read_csv(strat_file_path, header=None)

        # Extract the first 23 rows of the last column and the 7th to last column. 
        # The last column and the 7th to last column is where the best buyer and seller strategies will always be. 
        second_to_last_col = df.iloc[:24, -2]
        eighth_to_last_col = df.iloc[:24, -8]

        buyer_sum = eighth_to_last_col.sum()
        seller_sum = second_to_last_col.sum()
        print('')
        print(f'The sum of buyer strats for the first day is: {buyer_sum}')
        print(f'The sum of seller strats for the first day is: {seller_sum}')
        
        proof = buyer_sum + seller_sum
        print(f'The verification proof for the first in-market day of this market session is: {proof}')
        
        return proof
    
    
    # Adding a 'job' to the queue, this will be executed from the blockchain class. 
    def add_job_to_queue(self, job, job_queue):
        job_queue.put(job)
        
        

    # To account for possible missing parameters in a usrs JSON configs. 
    def merge_configs(self, user_config, random_config):
        merged_config = {}

        for key, value in random_config.items():
            if key in user_config:
                if isinstance(value, list) and isinstance(user_config[key], list):
                    merged_config[key] = user_config[key] + value
                else:
                    merged_config[key] = user_config[key]
            else:
                merged_config[key] = value

        return merged_config



    
    # Below three functions are for generating trader specs for different trader types. 
    # Their specificatons are different, so this was for PRSH, PRDE and all others seperately. 
    def generate_PRSH_config(self, seed):
        random.seed(seed)
        config = {
            'k': random.randint(4, 15),
            's_min': -1.0,
            's_max': 1.0
        }
        return config
    
    

    def generate_PRDE_config(self, seed):
        random.seed(seed)
        config = {
            'k': random.randint(4, 15),
            's_min': -1.0,
            's_max': 1.0,
            'f': random.uniform(0.0, 2.0),
            't': random.randint(7200, 14400)
        }
        
        return config
    
    
    
    
    

    def generate_trader_spec_config(self, seed, user_config=None, verifying=False):
        trader_types = ['PRZI', 'PRSH', 'PRDE', 'GVWY', 'ZIC', 'ZIP', 'SHVR', 'SNPR']

        traders_spec = {'sellers': [], 'buyers': []}
        
        if verifying:
            seed = self.seed

        for trader_role in ['buyers', 'sellers']:
            random.seed(seed)  # Reset seed for each market sesison. 
            seed += 1  # Increment seed for next iteration, where appropriate. 

            included_trader_types = [trader_type for trader_type in ['PRDE', 'PRSH'] if random.random() < 0.5]
            if not included_trader_types:
                included_trader_types.append(random.choice(['PRDE', 'PRSH']))
            included_trader_types += [trader_type for trader_type in trader_types if trader_type not in ['PRDE', 'PRSH'] and random.random() < 0.5]

    
            for trader_type in included_trader_types:
                num_traders = random.randint(2, 10)
                if trader_type in ['PRDE', 'PRSH', 'PRZI']:
                    config = self.generate_config(trader_type, seed)
                    traders_spec[trader_role].append((trader_type, num_traders, config))
                else:  # For 'GVWY', 'ZIC', 'ZIP', 'SHVR', 'SNPR'.
                    traders_spec[trader_role].append((trader_type, num_traders, None))  # Use None for the config. This is just a place holder, as some traders expect three parameters, even when nott applicable. 


        
        if user_config:
            traders_spec = self.merge_configs(user_config, traders_spec)

        return traders_spec
    
    
    

    def generate_config(self, trader_type, seed):
        random.seed(seed)
        if trader_type == 'PRDE':
            return self.generate_PRDE_config(seed)
        elif trader_type == 'PRSH':
            return self.generate_PRSH_config(seed)
        elif trader_type == 'PRZI':
            return {
                'k': 1,
                's_min': -1.0,
                's_max': 1.0
            }
        else:
            return None

    
    
    # Standard market specifications for BSE to run a market session. 
    # All of these can be changed in the JSON job request, EXCEPT end_time. 
    
    def generate_order_schedule_config(self, seed, user_config=None, verifying=False, end_time=60*60*24*25):
        if verifying:
            seed = self.seed
        random.seed(seed)

        
        sup_range_low = random.randint(20, 80)
        sup_range_high = random.randint(sup_range_low, 150) # Arbitrary range for supply and demand curves, largely for testing purposes. 
        dem_range_low = random.randint(sup_range_high, 200)
        dem_range_high = random.randint(dem_range_low, 300)

        sup_range = (sup_range_low, sup_range_high)
        dem_range = (dem_range_low, dem_range_high)
        
        #. testing
#         sup_range = (60, 60)
#         dem_range = (100, 100)

        start = 0
    
        self.market_time = end_time
        end = end_time
        
        stepmode = 'fixed'

        supply_schedule = [{'from': start, 'to': end, 'ranges': [sup_range], 'stepmode': stepmode}]
        demand_schedule = [{'from': start, 'to': end, 'ranges': [dem_range], 'stepmode': stepmode}]

        interval = 10 #random.randint(5, 360)
        timemode = 'periodic'

        order_sched = {'sup':supply_schedule, 'dem':demand_schedule, 'interval':interval, 'timemode':timemode}
        
        

        
        if user_config:
            order_sched = self.merge_configs(user_config, order_sched)

        return order_sched


 


    # This is the 'M' from the paper. 
    # Initially it was for solely running the useful computation of the protocol, but it got very messy because it was later used for the validation process. 
    # same_seed allows you to run market sessions of the same seed, in order to test if seeds actually make market sessions determinisitc. They do. 
    
    def run_market_sessions(self, n, target, pouw_difficulty, job_queue=None, same_seed=False, sup_dem_curve=False, posthash=False, user_config=None, verifying=False, end_time=60*60*24*25):
        trial_id_base = "trial_"
        trial_counter = 10001 # This was added so run market sessions did not overwite each other. 
        seed_counter = 1 

        for _ in range(n): 

            # If verifying, just set seed and user_config from block attributes. 
            if verifying:
                seed = self.seed
                user_config = self.user_config
            else:
                # Otherwise, continue with the usual logic. 
                if not same_seed or seed_counter == 0:
                    seed = self.pre_hash(target)
                    print(seed)
                else: 
                    print(f'Using the same seed: {seed}')

                if job_queue and not job_queue.empty():
                    user_config = job_queue.get()  # Get the job from the queue. 
                    print('A job has been popped off the queue, and will now be run.')

            # Generate the trader spec and market config for the given seed, using the current user_config if available. 
            order_schedule_config = self.generate_order_schedule_config(seed, user_config, verifying=verifying, end_time=end_time)
            traders_spec_config = self.generate_trader_spec_config(seed, user_config, verifying=verifying)

            print("Order Schedule Config:", order_schedule_config)
            print('Trader Specification Config:', traders_spec_config)

            from_time = order_schedule_config['sup'][0]['from']
            to_time = order_schedule_config['sup'][0]['to']

            trial_id = trial_id_base + str(trial_counter)
            print('The trial_id is', trial_id)
            
            # testing
            # print(n, trial_id, from_time, to_time, traders_spec_config, order_schedule_config)
            
            # Supply and demand curve functionality copied from BSE worksheet from the first term. 
            # For a time, I was getting 0 PPS for all traders, so I ran this to see if it was my supply and demand schedule causing the problem. 
            if sup_dem_curve:
                # Calculate the number of sellers and buyers
                seller_num = sum([num for _, num, _ in traders_spec_config['sellers']])
                buyer_num = sum([num for _, num, _ in traders_spec_config['buyers']])
                # Extract the supply and demand ranges
                sup_ranges = [order_schedule_config['sup'][0]['ranges'][0]]
                dem_ranges = [order_schedule_config['dem'][0]['ranges'][0]]
                # Extract the stepmode
                stepmode = order_schedule_config['sup'][0]['stepmode']
                # Plot and save the supply and demand curves
                print(f"sup_ranges: {sup_ranges}")
                print(f"dem_ranges: {dem_ranges}")
                self.plot_sup_dem(seller_num, sup_ranges, buyer_num, dem_ranges, stepmode)
                # plt.savefig(f'{trial_id}_sup_dem_curve.png')

            tdump = open(str(trial_id) + '_avg_balance.csv', 'w')

            BSE.market_session(trial_id, from_time, to_time, traders_spec_config, order_schedule_config, tdump, dump_all=False, verbose=False, session_seed=seed)

            tdump.close()

            print(f'Seed {seed} completed!')
            self.seed = seed
            self.user_config = user_config

            if posthash: # This conditional is here because when I was running this for data, I didn't want the posthashing step to run. A waste of comp. 
                strat_file_path = str(trial_id) + '_avg_balance.csv'
                is_acceptable = self.post_hash(strat_file_path, seed, pouw_difficulty, verifying)
                if is_acceptable:
                    print(f'The market session of seed: {seed} has passed the posthashing step!')
                    strat_file_path = str(trial_id) + '_strats.csv'
                    proof = self.extract_proof(strat_file_path)
                    self.proof = proof
                    return proof, seed
                else:
                    print(f"Market run with seed: {seed} rejected due to low PPS!")

            trial_counter += 1
            seed_counter += 1

        if posthash: # Only if posthashing was enabled, which for actual protocol runs, it will always be. 
            # If no session passed the posthash after all iterations
            print("No valid session found after all iterations.")
            return None, None

        print("All runs complete!")

        
        
        
    # If there are no job in the queue, then BSE Consensus will opt for standard PoW work done.
    def proof_of_work(self, difficulty):
        
        # The target string is a string of zeros with a length of the difficulty. 
        target = '0' * difficulty

        # Start with a nonce of zero, could be whatever. 
        self.nonce = 0

        # Calculate the initial hash, using the same method as the prehash. 
        self.hash = self.calculate_hash()

        # Keep hashing until we either find a valid hash or we decide to stop forr whatever reason. 
        while self.hash[:difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()

        print(f"Proof of Work completed with nonce value: {self.nonce} and resulting hash: {self.hash}")
        
        
        
        
        
    # This was needed for the supply and demand curves, but didn't relate to any block functionaltiy. Hence, it's a static method. 
    @staticmethod # Is static method, as it is not depedant on the class itself / self. 
    def getorderprice(i, sched, n, mode):
        pmin = min(sched[0][0], sched[0][1])
        pmax = max(sched[0][0], sched[0][1])
        prange = pmax - pmin
        stepsize = prange / (n - 1)
        halfstep = round(stepsize / 2.0)

        if mode == 'fixed':
            orderprice = pmin + int(i * stepsize)
        elif mode == 'jittered':
            orderprice = pmin + int(i * stepsize) + random.randint(-halfstep, halfstep)
        elif mode == 'random':
            if len(sched) > 1:
                s = random.randint(0, len(sched) - 1)
                pmin = min(sched[s][0], sched[s][1])
                pmax = max(sched[s][0], sched[s][1])
            orderprice = random.randint(pmin, pmax)
        return orderprice 

    @staticmethod
    def make_supply_demand_plot(bids, asks):
        volS = 0
        volB = 0

        fig, ax = plt.subplots()
        plt.ylabel('Price')
        plt.xlabel('Quantity')

        pr = 0
        for b in bids:
            if pr != 0:
                ax.plot([volB,volB], [pr,b], 'r-')
            line, = ax.plot([volB,volB+1], [b,b], 'r-')
            volB += 1
            pr = b
        if bids:
            line.set_label('Demand')

        pr = 0
        for s in asks:
            if pr != 0:
                ax.plot([volS,volS], [pr,s], 'b-')
            line, = ax.plot([volS,volS+1], [s,s], 'b-')
            volS += 1
            pr = s
        if asks:
            line.set_label('Supply')

        if bids or asks:
            plt.legend()
        plt.show()

    @staticmethod
    def plot_sup_dem(seller_num, sup_ranges, buyer_num, dem_ranges, stepmode):
        asks = []
        for s in range(seller_num):
            asks.append(Block.getorderprice(s, sup_ranges, seller_num, stepmode))
        asks.sort()
        bids = []
        for b in range(buyer_num):
            bids.append(Block.getorderprice(b, dem_ranges, buyer_num, stepmode))
        bids.sort()
        bids.reverse()

        Block.make_supply_demand_plot(bids, asks)
        
        
    


In [10]:

# Standard blockchain class, as I learnt from FTGP. 
# There is no complex network layer functionality, so this class is a lot simpler than the Block() class that handles to core of the consensus layer. 
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.pouw_difficulty = 50  # Posthash PPS requirment. Arbitrarily set to 50 at the moment, this is discussed more in the paper itself. 
        self.pow_difficulty = 2  # Number of leading zeros in the hash, standard PoW difficulty setting. 
        self.miners_queue = queue.Queue()  # For miners waiting to mine. Aim to implement this, but had diffiuclty as it's more network layer oriented. 
        self.job_queue = queue.Queue()

    def create_genesis_block(self):
        return Block(data="Genesis Block", previous_hash="0")

    def get_latest_block(self):
        return self.chain[-1]

    
    def add_block(self, block=None):     
        if not self.job_queue.empty():
            # If there are jobs in the queue, run BSE Consensus. 

            # Create a new block instance if one isn't provided, which will be the case when running a new instance / kernal. 
            if not block:
                block = Block()

            # Using BSE Consensus to mine a block. 
            proof, seed = block.run_market_sessions(n=1, target=1, pouw_difficulty=self.pouw_difficulty, job_queue=self.job_queue, same_seed=False, sup_dem_curve=False, posthash=True, end_time = 60*60*24*25)
            if proof is None or seed is None:
                print("Winning proof or seed not generated even after all iterations. Block not added.")
                return

            block.previous_hash = self.get_latest_block().hash
            block.hash = block.calculate_hash()
            self.chain.append(block)

        else:
            # If there are no jobs in the queue, run standard PoW instead. 
            if not block:
                block = Block()
            block.previous_hash = self.get_latest_block().hash
            block.proof_of_work(self.pow_difficulty)
            self.chain.append(block)

            

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

    def verify_block(self, block_index):
        if block_index >= len(self.chain) or block_index < 0:
            return f"Error: Block with index {block_index} does not exist."
        
        # Extract the block to be verified using the index specified when the method is called. 
        block_to_verify = self.chain[block_index]

        # Use the user_config stored in the block, necessary for validation. 
        user_config = block_to_verify.user_config
        print('')
        print(f'Verifying block of index {block_index}..')
        print('')
        
        # Adjust the end_time for verification to be one day, as opposed to 25 days. Validators do not runt he entire market session. 
        verification_end_time = 60*60*24 + 3600
        
        # Run the market session with the stored seed and configuration using the block_to_verify directly for the adjusted end_time, with posthashing. 
        # Bear in mind that the posthashing method accounts whether it's run for validation or not, so it won't unnecessarily check for aggregate PPS. 
        proof, _ = block_to_verify.run_market_sessions(n=1, target=1, pouw_difficulty=self.pouw_difficulty, same_seed=True, sup_dem_curve=False, posthash=True, user_config=block_to_verify.user_config, verifying=True, end_time=verification_end_time)

        # Compare the newly generated proof with the stored proof. 
        # The proofs must match to 8.d.p. 
        print(f'Miner proof: {block_to_verify.proof}')
        print(f'Validator proof {proof}')
        return block_to_verify.proof == proof
    
    
    
    # Purely a proof of concept fucntion. Without network metrics like hashrate / measure of network computation and/or blocktime, I'm not sure how youd implement this. 
    def increment_difficulties(self):
        
        self.pouw_difficulty += 10
        self.pow_difficulty += 1
        print(f"Updated PoUW Difficulty: {self.pouw_difficulty}")
        print(f"Updated PoW Difficulty: {self.pow_difficulty}")


        
        
    # Pass the job_queue when adding a job
    def add_job_to_queue(self, job):
        self.get_latest_block().add_job_to_queue(job, self.job_queue)
        
        
        
        
    # I had a lot of difficulty formatting my JSON input in a way that BSE would accept. 
    # BSE accepts configs in a very particular way, list of tuples etc.. 
    # These statics methods all the JSON job submissions to be handled properly. 
    @staticmethod
    def list_to_tuple(data):
        if isinstance(data, list):
            # Convert the list to a list of tuples
            return [tuple(item) if isinstance(item, list) else item for item in data]
        elif isinstance(data, dict):
            return {key: Blockchain.list_to_tuple(value) for key, value in data.items()}
        else:
            return data
        
        
        
        
    @staticmethod
    def load_and_process_config(json_input):
        user_config = json_input  # Directly use the input dictionary
        user_config['traders_spec']['sellers'] = Blockchain.list_to_tuple(user_config['traders_spec']['sellers'])
        user_config['traders_spec']['buyers'] = Blockchain.list_to_tuple(user_config['traders_spec']['buyers'])
        return user_config


In [11]:
# Testing the JSON user 'job' request functionality. 
json_input = {
    "traders_spec": {
        "sellers": [
            ["PRDE", 9, {
                "k": 15,
                "s_min": -1.0,
                "s_max": 1.0,
                "f": 0.73074910314467,
                "t": 12475
            }],
            ["PRSH", 6, {
                "k": 15,
                "s_min": -1.0,
                "s_max": 1.0
            }],
            ["GVWY", 7, None],
            ["ZIP", 4, None],
            ["SHVR", 6, None]
        ],
        "buyers": [
            ["PRDE", 3, {
                "k": 11,
                "s_min": -1.0,
                "s_max": 1.0,
                "f": 1.7527630534303504,
                "t": 11503
            }],
            ["PRSH", 10, {
                "k": 11,
                "s_min": -1.0,
                "s_max": 1.0
            }],
            ["PRZI", 8, {
                "k": 1,
                "s_min": -1.0,
                "s_max": 1.0
            }],
            ["GVWY", 9, None]
        ]
    },
    "order_schedule": {
        "sup": [{
            "from": 0,
            "to": 2160000,
            "ranges": [(24, 75)],
            "stepmode": "fixed"
        }],
        "dem": [{
            "from": 0,
            "to": 2160000,
            "ranges": [(78, 129)],
            "stepmode": "fixed"
        }],
        "interval": 10,
        "timemode": "periodic"
    }
}


# Initialise an instance of the Blockchain class. 
bc = Blockchain()


user_config = bc.load_and_process_config(json_input)
print(user_config)




# Add the user configuration as a job to the blockchain's job queue. 
bc.add_job_to_queue(user_config)

# Mine a new block using the added job configuration. 
bc.add_block()


{'traders_spec': {'sellers': [('PRDE', 9, {'k': 15, 's_min': -1.0, 's_max': 1.0, 'f': 0.73074910314467, 't': 12475}), ('PRSH', 6, {'k': 15, 's_min': -1.0, 's_max': 1.0}), ('GVWY', 7, None), ('ZIP', 4, None), ('SHVR', 6, None)], 'buyers': [('PRDE', 3, {'k': 11, 's_min': -1.0, 's_max': 1.0, 'f': 1.7527630534303504, 't': 11503}), ('PRSH', 10, {'k': 11, 's_min': -1.0, 's_max': 1.0}), ('PRZI', 8, {'k': 1, 's_min': -1.0, 's_max': 1.0}), ('GVWY', 9, None)]}, 'order_schedule': {'sup': [{'from': 0, 'to': 86400, 'ranges': [(24, 75)], 'stepmode': 'fixed'}], 'dem': [{'from': 0, 'to': 86400, 'ranges': [(78, 129)], 'stepmode': 'fixed'}], 'interval': 10, 'timemode': 'periodic'}}
3971594918
A job has been popped off the queue, and will now be run.
Order Schedule Config: {'sup': [{'from': 0, 'to': 259200, 'ranges': [(56, 90)], 'stepmode': 'fixed'}], 'dem': [{'from': 0, 'to': 259200, 'ranges': [(130, 157)], 'stepmode': 'fixed'}], 'interval': 10, 'timemode': 'periodic'}
Trader Specification Config: {'sel

In [14]:
# # Verify the newly mined block
is_valid = bc.verify_block(1)  # 1 is the index of the newly added block
print(f"Is the block valid?: {is_valid}")


Verifying block of index 1..

Order Schedule Config: {'sup': [{'from': 0, 'to': 90000, 'ranges': [(56, 90)], 'stepmode': 'fixed'}], 'dem': [{'from': 0, 'to': 90000, 'ranges': [(130, 157)], 'stepmode': 'fixed'}], 'interval': 10, 'timemode': 'periodic'}
Trader Specification Config: {'sellers': [('PRSH', 7, {'k': 14, 's_min': -1.0, 's_max': 1.0}), ('GVWY', 9, None), ('ZIP', 6, None), ('SHVR', 7, None)], 'buyers': [('PRSH', 7, {'k': 12, 's_min': -1.0, 's_max': 1.0}), ('PRZI', 3, {'k': 1, 's_min': -1.0, 's_max': 1.0}), ('ZIC', 10, None)]}
The trial_id is trial_10001
The main seed used for this market session is 3971594918
{'optimizer': 'PRSH', 'k': 12, 'strat_min': -1.0, 'strat_max': 1.0}
B00: PRSH active_strat=[0]:
[0]: s=-0.781611, start=0.000000, $=0.000000, pps=0.000000
[1]: s=-0.796642, start=0.000000, $=0.000000, pps=0.000000
[2]: s=-0.720602, start=0.000000, $=0.000000, pps=0.000000
[3]: s=-0.796281, start=0.000000, $=0.000000, pps=0.000000
[4]: s=-0.826460, start=0.000000, $=0.0000